In [1]:
#!/usr/bin/env python3
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
import en_core_web_sm

#gensim library: Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

#vizualization
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

#clustering
from sklearn.cluster import KMeans

#Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from sklearn.preprocessing import StandardScaler #scale arrays
import itertools
import multiprocessing #speed up computing
import os #saving & loading models

In [2]:
df_75 = pd.read_csv('procurement_enterprise_sg.csv')
df_75.shape

(254315, 6)

In [3]:
df_75.tail()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year
254310,311280,Conceptualisation And Design For URA 2019 Exhi...,Professional Services,Services,December,2018
254311,311281,"Term Contract For Supply, Installation And Mai...","Building, M&E Maintenance",Facilities Management,December,2018
254312,311282,Term Contract For Electrical Minor Works For M...,"Building, M&E Maintenance",Facilities Management,December,2018
254313,311283,Term Contract For Soil Investigation Works For...,Soil Investigation & Stabilization,Construction,December,2018
254314,311284,Renewal Of Backup Software Licences Maintenance,Softwares & Licences,IT & Telecommunication,December,2018


In [4]:
df_25 = pd.read_csv('procurement_enterprise_sg2.csv')
df_25.shape

(56969, 6)

In [5]:
df_25.tail()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year
56964,283755,Customer Satisfaction Survey 2012,Survey Services,Administration & Training,December,2012
56965,283756,"SURVEY OPERATIONS, DATA COLLECTION, DATA MANAG...",Survey Services,Administration & Training,December,2012
56966,283757,Office Rennovation,General Building & Minor Construction Works,Construction,November,2012
56967,283758,Provision of services to conduct surveys for C...,Survey Services,Administration & Training,December,2012
56968,283759,Provision of Custodian Services,Professional Services,Services,November,2012


In [6]:
# combine data
df = pd.concat([df_25, df_75], ignore_index=True)
df.shape

(311284, 6)

In [7]:
df.head()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year
0,1,Provision of Training Programme,Courses,Administration & Training,January,2012
1,2,Purchase of IT and AV products,"Ticketing, Travel Services, Tours & Excursions",Transportation,January,2012
2,3,Provision of foldable trapezium-shaped tables ...,Furniture,"Furniture, Office Equipment & Audio-Visual",January,2012
3,4,OYEA 2012 Advertisement in The Straits Times.,Advertising Services,Services,January,2012
4,5,Provision of Professional Services for the des...,Courses,Administration & Training,January,2012


In [8]:
df.tail()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year
311279,311280,Conceptualisation And Design For URA 2019 Exhi...,Professional Services,Services,December,2018
311280,311281,"Term Contract For Supply, Installation And Mai...","Building, M&E Maintenance",Facilities Management,December,2018
311281,311282,Term Contract For Electrical Minor Works For M...,"Building, M&E Maintenance",Facilities Management,December,2018
311282,311283,Term Contract For Soil Investigation Works For...,Soil Investigation & Stabilization,Construction,December,2018
311283,311284,Renewal Of Backup Software Licences Maintenance,Softwares & Licences,IT & Telecommunication,December,2018


In [9]:
df[56967:56970]

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year
56967,283758,Provision of services to conduct surveys for C...,Survey Services,Administration & Training,December,2012
56968,283759,Provision of Custodian Services,Professional Services,Services,November,2012
56969,52568,Engagement of Video-recording services for the...,Professional Services,Services,January,2013


In [10]:
df.describe()

,s/n,Award Year
count,311284.000000,311284.000000
mean,155642.500000,2014.467872
std,89860.094935,1.859220
min,1.000000,2012.000000
25%,77821.750000,2013.000000
50%,155642.500000,2014.000000
75%,233463.250000,2016.000000
max,311284.000000,2018.000000


In [11]:
# consolidate 2012-2018
# df.to_csv('procurement_enterprise_sg3.csv', index=False)

In [12]:
#df with text (quotation combined with category)
pd.set_option('display.max_colwidth', -1)
df1 = pd.DataFrame({'s/n':df['s/n'],'text': df['Quotation Description']+ ' '+ df['Category']})
# df_text = df_text.dropna().drop_duplicates()
df1.shape

(311284, 2)

In [13]:
df1.head()

,s/n,text
0,1,Provision of Training Programme Courses
1,2,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions"
2,3,Provision of foldable trapezium-shaped tables with casters Furniture
3,4,OYEA 2012 Advertisement in The Straits Times. Advertising Services
4,5,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses


In [14]:
df1.isnull().sum()

s/n     0
text    5
dtype: int64

In [154]:
df2 = df1[['text']].drop_duplicates().dropna().reset_index(drop=True)
df2.isnull().sum()

text    0
dtype: int64

In [155]:
df2.shape

(291208, 1)

In [23]:
# import en_core_web_sm
nlp = en_core_web_sm.load(disable=['ner', 'parser'])

In [24]:
# doc needs to be a spacy Doc object
def cleaning(doc):
    # Lemmatizes and removes stopwords (allow others)
    txt = [token.lemma_ for token in doc if not token.is_stop] #...and token is not 'others'
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [25]:
# Removes non-alphabetic characters:
brief_cleaning = (re.sub("[^A-Za-z_']+", ' ', str(row)).lower() for row in df2['text'])

In [26]:
bf_list = list(brief_cleaning)

In [27]:
bf_list[:5]

['provision of training programme courses',
 'purchase of it and av products ticketing travel services tours excursions',
 'provision of foldable trapezium shaped tables with casters furniture',
 'oyea advertisement in the straits times advertising services',
 'provision of professional services for the design and delivery of a strategic alignment workshop for leaders of academy of singapore teachers ast courses']

In [28]:
# Clean text
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(bf_list, batch_size=5000, n_threads=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 10.05 mins


In [29]:
len(txt)

291208

In [156]:
df2['text2'] = txt

In [298]:
# .drop_duplicates().dropna().reset_index(drop=True)
df2= df2.dropna().drop_duplicates().reset_index(drop=True)
df2.shape, df2.isnull().sum()

((291020, 2), text     0
 text2    0
 dtype: int64)

In [299]:
df2.head()

,text,text2
0,Provision of Training Programme Courses,provision training programme course
1,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion
2,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture
3,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service
4,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course


In [158]:
df2.shape

(291208, 2)

In [34]:
df_word = pd.read_csv('procurement_word2vec_output_4.csv')
df_word.shape

(100045, 10)

In [35]:
df_word1 = df_word[df_word.is_target_word==1][['word','cluster_3']]
df_word1.shape

(9095, 2)

In [36]:
df_word1.head()

,word,cluster_3
0,engagement,5
11,video_recording,9
22,service,5
33,seminar,113
44,jan,9999


In [37]:
word_formatted = []
for word in df_word1.word:
    if '_' in word:
        formatted_word = word.replace('_',' ')
        word_formatted.append(formatted_word)
    else:
        word_formatted.append(word)
len(word_formatted)

9095

In [38]:
df_word1.insert(1, "word_2", word_formatted)

In [39]:
df_word1.head()

,word,word_2,cluster_3
0,engagement,engagement,5
11,video_recording,video recording,9
22,service,service,5
33,seminar,seminar,113
44,jan,jan,9999


In [47]:
df_word2 = df_word1[(df_word1.word == df_word1.word_2) & (df_word1.cluster_3 != 9999)]
df_word2 = df_word2[['word_2','cluster_3']].reset_index(drop=True)
df_word2.shape

(5185, 2)

In [48]:
df_word2.head()

,word_2,cluster_3
0,engagement,5
1,service,5
2,seminar,113
3,supply,2
4,install,128


In [67]:
#cluster meaning
cluster_dict = {
    0: 'Specialised software',
    1: 'Non-Engineering Services',
    2: 'School-related purchases',
    3: 'Events',
    4: 'Laboratory Equipments',
    5: 'Professional Advisory Services',
    6: 'Community Development',
    7: 'Equipments',
    8: 'Security and Facility Management Services',
    9: 'Courses',
    102: 'Outdoor Services/Nature',
    104: 'Laboratory Equipments',
    105: 'Printed Media',
    106: 'IT Software/License',
    107: 'Travel',
    108: 'Urban Development',
    109: 'Courses',
    111: 'Music Equipments/Trainers',
    113: 'Events',
    120: 'Home Appliances',
    124: 'School CCA',
    125: 'Engineering Services/Equipments',
    127: 'Office Supplies/IT Equipments',
    128: 'Renovation'
#     9999: 'To Remove'    
}
len(cluster_dict)

24

In [68]:
df_cluster_dict = pd.DataFrame.from_dict(cluster_dict, orient='index').reset_index()
df_cluster_dict.columns = ['cluster_3', 'cluster_meaning']

In [69]:
df_cluster_dict

,cluster_3,cluster_meaning
0,0,Specialised software
1,1,Non-Engineering Services
2,2,School-related purchases
3,3,Events
4,4,Laboratory Equipments
5,5,Professional Advisory Services
6,6,Community Development
7,7,Equipments
8,8,Security and Facility Management Services
9,9,Courses


In [70]:
df_word3 = pd.merge(df_word2, df_cluster_dict, how='left', on=['cluster_3'])

In [71]:
df_word3.shape

(5185, 3)

In [72]:
df_word3.head()

,word_2,cluster_3,cluster_meaning
0,engagement,5,Professional Advisory Services
1,service,5,Professional Advisory Services
2,seminar,113,Events
3,supply,2,School-related purchases
4,install,128,Renovation


In [55]:
df_word3[df_word3.word_2 == 'design production']

,word_2,cluster_3,cluster_meaning


In [73]:
df_word3[df_word3['word_2'].str.contains("service")].sort_values(by=['cluster_meaning'])

,word_2,cluster_3,cluster_meaning
1,service,5,Professional Advisory Services


In [112]:
def cluster(word):
    return df_word3[df_word3.word_2==word].cluster_meaning.tolist()[0] 
#     return df_word3[df_word3.word_2==word].cluster_meaning.tolist()[0]+' ('+str(df_word3[df_word3.word_2==word].cluster_3.tolist()[0])+')' 
word = 'service'
cluster(word)

'Professional Advisory Services'

In [111]:
def cluster_set(text):
    return ", ".join(sorted(list(set([cluster(word) for word in text.split() if word in df_word3.word_2.tolist()]))))

text = 'oyea provision design production delivery food nutrition teaching learning guide printing service jan'
cluster_set(text)

'Community Development, Courses, Events, Laboratory Equipments, Printed Media, Professional Advisory Services, School-related purchases'

In [161]:
df2.shape

(291208, 2)

In [162]:
df_test = df2[:1000]
df_test.shape

(1000, 2)

In [163]:
input_list = df_test.text2.tolist()
len(input_list)

1000

In [281]:
# cluster text
t = time()
cluster_sets = [cluster_set(str(doc)) for doc in nlp.pipe(input_list, batch_size=5000, n_threads=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

KeyboardInterrupt: 

In [165]:
cluster_sets[-1]

'Music Equipments/Trainers, Professional Advisory Services'

In [166]:
input_list[-1]

'hougang secondary school gamelan instructor period jan december professional service'

In [167]:
df_word3[df_word3.word_2 == 'gamelan']

,word_2,cluster_3,cluster_meaning
548,gamelan,111,Music Equipments/Trainers


In [168]:
df_test.tail(1)

,text,text2
999,Hougang Secondary School - Gamelan Instructor. Period: 3 Jan 2012 to 31 December 2012 Professional Services,hougang secondary school gamelan instructor period jan december professional service


In [120]:
df_word_all = pd.merge(df_word1, df_cluster_dict, how='left', on=['cluster_3'])

In [150]:
df_word_all[df_word_all['word_2'].str.contains("advertising")].sort_values(by=['cluster_meaning'])

,word,word_2,cluster_3,cluster_meaning
1158,gamelan,gamelan,111,Music Equipments/Trainers


In [105]:
df_test['cluster_sets'] = cluster_sets

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
df_test.shape, len(cluster_sets)

((500, 3), 500)

In [107]:
df_test.head()

,text,text2,cluster_sets
0,Provision of Training Programme Courses,provision training programme course,"Community Development, School-related purchases"
1,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion,"Laboratory Equipments, Office Supplies/IT Equipments, Professional Advisory Services, Travel"
2,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture,"Home Appliances, Renovation, School-related purchases"
3,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service,"Courses, Printed Media, Professional Advisory Services, Travel"
4,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course,"Community Development, Courses, Laboratory Equipments, Non-Engineering Services, Printed Media, Professional Advisory Services, School-related purchases"


In [122]:
df_word_all.shape

(9095, 4)

In [295]:
df2.shape[0]

291020

In [297]:
583*500

291500

In [304]:
#batch data at 1000 per pass
t = time()
batching_size = 500
input_list = df2.text2.tolist()
no_of_batches = int(np.ceil(len(input_list)/batching_size))
no_of_batches
cluster_set_list = []
for i in range(no_of_batches):
    cluster_set_list+= [cluster_set(str(doc)) for doc in nlp.pipe(input_list[i*batching_size:(i+1)*batching_size], batch_size=5000, n_threads=-1)]
    print('Time to cluster for {}th iterations: {} mins'.format(i,round((time() - t) / 60, 2)),'|data left:',len(input_list)-(i+1)*batching_size)

Time to cluster for 0th iterations: 0.18 mins |data left: 290520
Time to cluster for 1th iterations: 0.33 mins |data left: 290020
Time to cluster for 2th iterations: 0.48 mins |data left: 289520
Time to cluster for 3th iterations: 0.65 mins |data left: 289020
Time to cluster for 4th iterations: 0.76 mins |data left: 288520
Time to cluster for 5th iterations: 0.96 mins |data left: 288020
Time to cluster for 6th iterations: 1.07 mins |data left: 287520
Time to cluster for 7th iterations: 1.25 mins |data left: 287020
Time to cluster for 8th iterations: 1.39 mins |data left: 286520
Time to cluster for 9th iterations: 1.57 mins |data left: 286020
Time to cluster for 10th iterations: 1.72 mins |data left: 285520
Time to cluster for 11th iterations: 1.88 mins |data left: 285020
Time to cluster for 12th iterations: 2.02 mins |data left: 284520
Time to cluster for 13th iterations: 2.22 mins |data left: 284020
Time to cluster for 14th iterations: 2.35 mins |data left: 283520
Time to cluster for 

Time to cluster for 124th iterations: 21.02 mins |data left: 228520
Time to cluster for 125th iterations: 21.16 mins |data left: 228020
Time to cluster for 126th iterations: 21.31 mins |data left: 227520
Time to cluster for 127th iterations: 21.46 mins |data left: 227020
Time to cluster for 128th iterations: 21.62 mins |data left: 226520
Time to cluster for 129th iterations: 21.8 mins |data left: 226020
Time to cluster for 130th iterations: 21.96 mins |data left: 225520
Time to cluster for 131th iterations: 22.17 mins |data left: 225020
Time to cluster for 132th iterations: 22.29 mins |data left: 224520
Time to cluster for 133th iterations: 22.47 mins |data left: 224020
Time to cluster for 134th iterations: 22.59 mins |data left: 223520
Time to cluster for 135th iterations: 22.74 mins |data left: 223020
Time to cluster for 136th iterations: 22.88 mins |data left: 222520
Time to cluster for 137th iterations: 23.05 mins |data left: 222020
Time to cluster for 138th iterations: 23.23 mins 

Time to cluster for 245th iterations: 60.68 mins |data left: 168020
Time to cluster for 246th iterations: 60.8 mins |data left: 167520
Time to cluster for 247th iterations: 60.92 mins |data left: 167020
Time to cluster for 248th iterations: 61.05 mins |data left: 166520
Time to cluster for 249th iterations: 61.16 mins |data left: 166020
Time to cluster for 250th iterations: 61.28 mins |data left: 165520
Time to cluster for 251th iterations: 61.39 mins |data left: 165020
Time to cluster for 252th iterations: 61.5 mins |data left: 164520
Time to cluster for 253th iterations: 61.64 mins |data left: 164020
Time to cluster for 254th iterations: 61.77 mins |data left: 163520
Time to cluster for 255th iterations: 61.88 mins |data left: 163020
Time to cluster for 256th iterations: 62.0 mins |data left: 162520
Time to cluster for 257th iterations: 62.12 mins |data left: 162020
Time to cluster for 258th iterations: 62.24 mins |data left: 161520
Time to cluster for 259th iterations: 62.39 mins |d

Time to cluster for 365th iterations: 388.37 mins |data left: 108020
Time to cluster for 366th iterations: 388.5 mins |data left: 107520
Time to cluster for 367th iterations: 388.62 mins |data left: 107020
Time to cluster for 368th iterations: 388.73 mins |data left: 106520
Time to cluster for 369th iterations: 388.84 mins |data left: 106020
Time to cluster for 370th iterations: 388.94 mins |data left: 105520
Time to cluster for 371th iterations: 389.05 mins |data left: 105020
Time to cluster for 372th iterations: 389.16 mins |data left: 104520
Time to cluster for 373th iterations: 389.26 mins |data left: 104020
Time to cluster for 374th iterations: 389.38 mins |data left: 103520
Time to cluster for 375th iterations: 389.5 mins |data left: 103020
Time to cluster for 376th iterations: 389.62 mins |data left: 102520
Time to cluster for 377th iterations: 389.73 mins |data left: 102020
Time to cluster for 378th iterations: 389.84 mins |data left: 101520
Time to cluster for 379th iterations

Time to cluster for 486th iterations: 574.26 mins |data left: 47520
Time to cluster for 487th iterations: 574.37 mins |data left: 47020
Time to cluster for 488th iterations: 574.47 mins |data left: 46520
Time to cluster for 489th iterations: 574.59 mins |data left: 46020
Time to cluster for 490th iterations: 574.69 mins |data left: 45520
Time to cluster for 491th iterations: 574.81 mins |data left: 45020
Time to cluster for 492th iterations: 574.92 mins |data left: 44520
Time to cluster for 493th iterations: 575.02 mins |data left: 44020
Time to cluster for 494th iterations: 575.12 mins |data left: 43520
Time to cluster for 495th iterations: 575.23 mins |data left: 43020
Time to cluster for 496th iterations: 575.35 mins |data left: 42520
Time to cluster for 497th iterations: 575.47 mins |data left: 42020
Time to cluster for 498th iterations: 575.57 mins |data left: 41520
Time to cluster for 499th iterations: 575.68 mins |data left: 41020
Time to cluster for 500th iterations: 575.8 mins

In [294]:
(no_of_batches)*batching_size- len(input_list)

480

In [351]:
637.03/60

10.617166666666666

In [305]:
cluster_set_list[-1]

'IT Software/License'

In [306]:
input_list[-1]

'renewal backup software licence maintenance software licence'

In [307]:
len(cluster_set_list)

291020

In [309]:
df2['cluster_set'] = cluster_set_list

In [310]:
df2.shape

(291020, 3)

In [311]:
df2.head()

,text,text2,cluster_set
0,Provision of Training Programme Courses,provision training programme course,"Community Development, School-related purchases"
1,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion,"Laboratory Equipments, Office Supplies/IT Equipments, Professional Advisory Services, Travel"
2,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture,"Home Appliances, Renovation, School-related purchases"
3,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service,"Courses, Printed Media, Professional Advisory Services, Travel"
4,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course,"Community Development, Courses, Laboratory Equipments, Non-Engineering Services, Printed Media, Professional Advisory Services, School-related purchases"


In [312]:
df2.tail()

,text,text2,cluster_set
291015,"Tender For The Distribution Of 500 Copies Of ""Greening The Vertical Garden City"" And 500 Copies Of ""Vertical Garden City: Singapore"" Publications Publications & Periodicals",tender distribution copy green vertical garden city copy vertical garden city singapore publication publication periodical,"Events, Home Appliances, Outdoor Services/Nature, Printed Media, Travel"
291016,Conceptualisation And Design For URA 2019 Exhibition Professional Services,conceptualisation design ura exhibition professional service,"Courses, Events, Printed Media, Professional Advisory Services"
291017,"Term Contract For Supply, Installation And Maintenance Of Solar Lighted Signboards At Vehicle Parks For A Period Of Three Years With An Option To Extend For Two Years. Building, M&E Maintenance",term contract supply installation maintenance solar lighted signboard vehicle park period year option extend year build m e maintenance,"IT Software/License, Laboratory Equipments, Non-Engineering Services, Renovation, School-related purchases"
291018,"Term Contract For Electrical Minor Works For Maintenance Of Lighting Systems In Vehicle Parks For A Period Of Three (3) Years With An Option To Extend For Two (2) Years Building, M&E Maintenance",term contract electrical minor work maintenance light system vehicle park period year option extend year build m e maintenance,"Engineering Services/Equipments, IT Software/License, Laboratory Equipments, Non-Engineering Services, Renovation, Urban Development"
291019,Renewal Of Backup Software Licences Maintenance Softwares & Licences,renewal backup software licence maintenance software licence,IT Software/License


In [313]:
#save cluster_set
df2.to_csv('procurement_enterprise_sg4.csv', index=False)

In [328]:
df_test.tail()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year,text
95,96,SCSS: Applications are invited to supply a Percussion Tutor for School Brass Band. Schedule: Every Friday from 1300 hrs to 1830 hrs. Period: 1 Jan 2012 to 31 Dec 2012. Candidates to be suitably qualified and have vast experience with school bands.,"Data Entry, Supply of Manpower Services",Services,January,2012,"SCSS: Applications are invited to supply a Percussion Tutor for School Brass Band. Schedule: Every Friday from 1300 hrs to 1830 hrs. Period: 1 Jan 2012 to 31 Dec 2012. Candidates to be suitably qualified and have vast experience with school bands. Data Entry, Supply of Manpower Services"
96,97,Swimming CCA Coach - 2012,Professional Services,Services,January,2012,Swimming CCA Coach - 2012 Professional Services
97,98,Badminton Recreational - 2012,"Data Entry, Supply of Manpower Services",Services,January,2012,"Badminton Recreational - 2012 Data Entry, Supply of Manpower Services"
98,99,CLDDS Audio Broadcasting & News Reporting CCA 2012,"AV Equipment, Photographic Equipment & Accessories","Furniture, Office Equipment & Audio-Visual",January,2012,"CLDDS Audio Broadcasting & News Reporting CCA 2012 AV Equipment, Photographic Equipment & Accessories"
99,100,Badminton CCA (School Team) 2012,"Data Entry, Supply of Manpower Services",Services,January,2012,"Badminton CCA (School Team) 2012 Data Entry, Supply of Manpower Services"


In [322]:
df3 = pd.merge(df, df1, how='left', on=['s/n'])

In [323]:
df3.head()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year,text
0,1,Provision of Training Programme,Courses,Administration & Training,January,2012,Provision of Training Programme Courses
1,2,Purchase of IT and AV products,"Ticketing, Travel Services, Tours & Excursions",Transportation,January,2012,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions"
2,3,Provision of foldable trapezium-shaped tables with casters,Furniture,"Furniture, Office Equipment & Audio-Visual",January,2012,Provision of foldable trapezium-shaped tables with casters Furniture
3,4,OYEA 2012 Advertisement in The Straits Times.,Advertising Services,Services,January,2012,OYEA 2012 Advertisement in The Straits Times. Advertising Services
4,5,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST).,Courses,Administration & Training,January,2012,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses


In [330]:
df_test = df3.head(10000)

In [334]:
df4 = pd.merge(df3, df2, how='left', on=['text'])

In [335]:
df4.shape

(311284, 9)

In [336]:
df4.head()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year,text,text2,cluster_set
0,1,Provision of Training Programme,Courses,Administration & Training,January,2012,Provision of Training Programme Courses,provision training programme course,"Community Development, School-related purchases"
1,2,Purchase of IT and AV products,"Ticketing, Travel Services, Tours & Excursions",Transportation,January,2012,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion,"Laboratory Equipments, Office Supplies/IT Equipments, Professional Advisory Services, Travel"
2,3,Provision of foldable trapezium-shaped tables with casters,Furniture,"Furniture, Office Equipment & Audio-Visual",January,2012,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture,"Home Appliances, Renovation, School-related purchases"
3,4,OYEA 2012 Advertisement in The Straits Times.,Advertising Services,Services,January,2012,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service,"Courses, Printed Media, Professional Advisory Services, Travel"
4,5,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST).,Courses,Administration & Training,January,2012,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course,"Community Development, Courses, Laboratory Equipments, Non-Engineering Services, Printed Media, Professional Advisory Services, School-related purchases"


In [337]:
df4.tail()

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year,text,text2,cluster_set
311279,311280,Conceptualisation And Design For URA 2019 Exhibition,Professional Services,Services,December,2018,Conceptualisation And Design For URA 2019 Exhibition Professional Services,conceptualisation design ura exhibition professional service,"Courses, Events, Printed Media, Professional Advisory Services"
311280,311281,"Term Contract For Supply, Installation And Maintenance Of Solar Lighted Signboards At Vehicle Parks For A Period Of Three Years With An Option To Extend For Two Years.","Building, M&E Maintenance",Facilities Management,December,2018,"Term Contract For Supply, Installation And Maintenance Of Solar Lighted Signboards At Vehicle Parks For A Period Of Three Years With An Option To Extend For Two Years. Building, M&E Maintenance",term contract supply installation maintenance solar lighted signboard vehicle park period year option extend year build m e maintenance,"IT Software/License, Laboratory Equipments, Non-Engineering Services, Renovation, School-related purchases"
311281,311282,Term Contract For Electrical Minor Works For Maintenance Of Lighting Systems In Vehicle Parks For A Period Of Three (3) Years With An Option To Extend For Two (2) Years,"Building, M&E Maintenance",Facilities Management,December,2018,"Term Contract For Electrical Minor Works For Maintenance Of Lighting Systems In Vehicle Parks For A Period Of Three (3) Years With An Option To Extend For Two (2) Years Building, M&E Maintenance",term contract electrical minor work maintenance light system vehicle park period year option extend year build m e maintenance,"Engineering Services/Equipments, IT Software/License, Laboratory Equipments, Non-Engineering Services, Renovation, Urban Development"
311282,311283,Term Contract For Soil Investigation Works For A Period Of Two (2) Years With An Option To Extend For Another One (1) Year,Soil Investigation & Stabilization,Construction,December,2018,Term Contract For Soil Investigation Works For A Period Of Two (2) Years With An Option To Extend For Another One (1) Year Soil Investigation & Stabilization,term contract soil investigation work period year option extend year soil investigation stabilization,"Outdoor Services/Nature, Professional Advisory Services, Urban Development"
311283,311284,Renewal Of Backup Software Licences Maintenance,Softwares & Licences,IT & Telecommunication,December,2018,Renewal Of Backup Software Licences Maintenance Softwares & Licences,renewal backup software licence maintenance software licence,IT Software/License


In [340]:
df4.isnull().sum()

s/n                      0  
Quotation Description    0  
Category                 5  
Main Category            0  
Award Month              0  
Award Year               0  
text                     5  
text2                    285
cluster_set              285
dtype: int64

In [343]:
df_dup = df4[['text']]
df_dup[df_dup.duplicated()]

,text
192,"Basketball Coach 2012 Data Entry, Supply of Manpower Services"
403,Invitation To Quote: Co-Designing of Learning Package for Programme for Active Learning (PAL) Visual Arts Courses
483,Requirement : 1) Refer to attached document for program specific requirements. 2) Weekly 1 hr- 1 hr 15 mins session (8 hrs - 10 hrs Module to be completed within 5 Weeks. 3) 1 Trainer:20 Students ratio must be met at all times. 4) Identified Competencies/Outcomes must be assessed by vendor. 5) Note : there is only one Mac Lab. 6) Apps designed by students must be published on Apples iTune Store for iPad/iPhone use. Courses
485,Requirement : 1) Refer to attached document for prog specific requirement. 2) Weekly 1 hr-1 hr 15 mins session (5 hrs- 6 hrs 15 mins Module to be completed within 5 Weeks. 3) 1 Trainer:20 Students ratio must be met at all times. 4) Identified Competencies/Outcomes must be assessed by vendor. 5) Note:There is only one Mac Lab. 6) Apps designed by students must be published on Apples iTune Store for iPad/iPhone use. Courses
820,"Basketball Girls Intructor for 2012. To quote price base on hourly rate. To conduct Basketball CCA training every Monday and Wednesday 3.30pm-5pm. Subject to changes according to school plan. Data Entry, Supply of Manpower Services"
1390,Invitation to Quote - Supply and installation of Galvanised Fencing with 5-layer net and Concrete Footing for training of Track & Field throwing events. Fencing & Ironworks
1407,"QUOTATION FOR SUPPLY OF INSTRUCTOR FOR MUSIC CURRICULUM (AMIS) FOR PEI HWA PRESBYTERIAN PRIMARY SCHOOL Data Entry, Supply of Manpower Services"
1454,Symphonic Band Professional Services
1535,Innova Junior College - Supply of Basketball Instructor for our Basketball CCA for the period from 1 January 2012 to 31 December 2012 with an option to extend the contract for another year in 2013. Professional Services
1571,Concert Band (SYF) training for Y2012 for St.Anthony's Canossian Pri School. Courses


In [347]:
df4.text.value_counts(ascending=False)

Provision of Event Management Services  Event Organising                                                                                                                                                                                                                                                                                                                                                                                 42
#NAME?  Courses                                                                                                                                                                                                                                                                                                                                                                                                                          34
Supply and Delivery of Forms  Printing Services                                                                                                 

In [350]:
#save serial no. mapping
#save cluster_set
df4.to_csv('procurement_enterprise_sg5.csv', index=False)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('procurement_enterprise_sg5.csv')

In [4]:
df.shape

(311284, 9)

In [7]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Category         5
text             5
text2          285
cluster_set    726
dtype: int64

In [10]:
df[df["text"].isnull()]

,s/n,Quotation Description,Category,Main Category,Award Month,Award Year,text,text2,cluster_set
53247,280038,DRUG TRACE DETECTORS,NaN,Unknown Category,February,2012,NaN,NaN,NaN
112213,107812,Re-certification of SS506 and 2 surveillance a...,NaN,Unknown Category,January,2014,NaN,NaN,NaN
294949,294950,Application Development and Maintenance Servic...,NaN,Unknown Category,March,2015,NaN,NaN,NaN
297588,297589,Supply and Maintenance of Laboratory Analytica...,NaN,Unknown Category,September,2015,NaN,NaN,NaN
306479,306480,Travel Related Services,NaN,Unknown Category,October,2017,NaN,NaN,NaN


In [119]:
t = time()
clusters_in_text = []
# clusters_in_text_meaning = []
for text in df_test.text2:
    clusters = []
    if any(word in text.split() for word in df_word2.word_2):
#         print(text)
        for i in range(len(df_word2)):
            word = df_word2.iloc[i,0]
            
            if len(word.split())>1:
                if (word in text) & (df_word2.iloc[i,1]!=9999):
                    print(word, '|',text, df_word2.iloc[i,1])
                    clusters.append(df_word2.iloc[i,1])
            else:   
                if (word in text.split()) & (df_word2.iloc[i,1]!=9999):
                    clusters.append(df_word2.iloc[i,1])
                    
    clusters = sorted(list(set(clusters)))
    clusters_2 = ", ".join(str(v) for v in clusters)
    clusters_in_text.append(clusters_2)
    
df_test['cluster_set'] = clusters_in_text
print('Time to do mapping: {} mins'.format(round((time() - t) / 60, 2)))
df_test

ticket travel service | purchase av product ticket travel service tour excursion 107
ticket travel servic | purchase av product ticket travel service tour excursion 107
travel service | purchase av product ticket travel service tour excursion 107
advertisement strait | oyea advertisement strait time advertising service 105
advertising service | oyea advertisement strait time advertising service 9
strait time | oyea advertisement strait time advertising service 105
professional service | provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course 3
professional servic | provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course 3
strategic planning | strategic planning ssd induction programme course 6
printing service | provision design production delivery food nutrition teaching learning guide printing service 9
teaching learning | provision design production deliver

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,s/n,text,text2,cluster_set
0,1,Provision of Training Programme Courses,provision training programme course,"2, 6"
1,2,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion,"4, 5, 107, 127"
2,3,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture,"2, 120, 128"
3,4,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service,"5, 9, 105, 107"
4,5,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course,"1, 2, 3, 4, 5, 6, 9, 104, 105, 109"
5,6,Strategic Planning - SSD Induction Programme. Courses,strategic planning ssd induction programme course,"6, 9, 127"
6,7,Staff Development Systems and Processes - SSD Induction Programme. Courses,staff development system process ssd induction programme course,"4, 5, 9, 113, 127"
7,8,Provision of Training Services for Teacher Preparatory Programme (TPP) in 2012. Courses,provision training service teacher preparatory programme tpp course,"2, 5, 6, 109"
8,9,Quarterly Inspection and Annual Certification of Challenge Ropes Courses and Climbing Walls of MOE Labrador Adventure Centre Sports & Recreational Equipment,quarterly inspection annual certification challenge rope course climb wall moe labrador adventure centre sport recreational equipment,"3, 5, 6, 7, 102, 108, 120, 124, 128"
9,10,"Provision for Design, Production and Delivery of Food & Nutrition Teaching and Learning Guides. Printing Services",provision design production delivery food nutrition teaching learning guide printing service,"2, 4, 5, 6, 9, 105, 109, 113"


In [182]:
t = time()

clusters_in_text = []
# clusters_in_text_meaning = []
for text in df_test.text2:
    clusters = []
    if any(word in text.split() for word in df_word3.word_2):
#         print(text) multithread?
        for i in range(len(df_word3)):
            word = df_word3.iloc[i,0]
            if len(word.split())>1:
                if (word in text) & (df_word3.iloc[i,2]!='To Remove'):
#                     print(word, '|',text,'|', df_word3.iloc[i,2])
                    clusters.append(df_word3.iloc[i,2])
            else:   
                if (word in text.split()) & (df_word3.iloc[i,2]!='To Remove'):
                    clusters.append(df_word3.iloc[i,2])
                    
    clusters = sorted(list(set(clusters)))
    clusters_2 = ", ".join(str(v) for v in clusters)
    clusters_in_text.append(clusters_2)
    
df_test['cluster_set'] = clusters_in_text
print('Time to do mapping: {} mins'.format(round((time() - t) / 60, 2)))
df_test

Time to do mapping: 0.32 mins


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,s/n,text,text2,cluster_set
0,1,Provision of Training Programme Courses,provision training programme course,"Awareness & Support, Outdoor"
1,2,"Purchase of IT and AV products Ticketing, Travel Services, Tours & Excursions",purchase av product ticket travel service tour excursion,"Corporate Language, Inspection Works & Tools, Office Supplies/IT Equipments, Travel"
2,3,Provision of foldable trapezium-shaped tables with casters Furniture,provision foldable trapezium shaped table caster furniture,"Home Appliances, Outdoor, Renovation"
3,4,OYEA 2012 Advertisement in The Straits Times. Advertising Services,oyea advertisement strait time advertising service,"Corporate Language, Media Coverage, Printed Media, Travel"
4,5,Provision of Professional Services for the design and delivery of a strategic alignment workshop for leaders of Academy of Singapore Teachers (AST). Courses,provision professional service design delivery strategic alignment workshop leader academy singapore teacher ast course,"Awareness & Support, Corporate Language, Courses/Training, Daily Activities & Utilities, Events, Inspection Works & Tools, Media Coverage, Medical/Laboratory Equipment, Outdoor, Printed Media"
5,6,Strategic Planning - SSD Induction Programme. Courses,strategic planning ssd induction programme course,"Awareness & Support, Media Coverage, Office Supplies/IT Equipments"
6,7,Staff Development Systems and Processes - SSD Induction Programme. Courses,staff development system process ssd induction programme course,"Corporate Language, Events, Inspection Works & Tools, Media Coverage, Office Supplies/IT Equipments"
7,8,Provision of Training Services for Teacher Preparatory Programme (TPP) in 2012. Courses,provision training service teacher preparatory programme tpp course,"Awareness & Support, Corporate Language, Courses/Training, Outdoor"
8,9,Quarterly Inspection and Annual Certification of Challenge Ropes Courses and Climbing Walls of MOE Labrador Adventure Centre Sports & Recreational Equipment,quarterly inspection annual certification challenge rope course climb wall moe labrador adventure centre sport recreational equipment,"Awareness & Support, Corporate Language, Events, Home Appliances, Inspection Works & Tools, Outdoor Services/Nature, Renovation, School CCA, Urban Development"
9,10,"Provision for Design, Production and Delivery of Food & Nutrition Teaching and Learning Guides. Printing Services",provision design production delivery food nutrition teaching learning guide printing service,"Awareness & Support, Corporate Language, Courses/Training, Events, Inspection Works & Tools, Media Coverage, Outdoor, Printed Media"


In [250]:
df2.tail()

,s/n,text,text2
311279,311280,Conceptualisation And Design For URA 2019 Exhibition Professional Services,conceptualisation design ura exhibition professional service
311280,311281,"Term Contract For Supply, Installation And Maintenance Of Solar Lighted Signboards At Vehicle Parks For A Period Of Three Years With An Option To Extend For Two Years. Building, M&E Maintenance",term contract supply installation maintenance solar lighted signboard vehicle park period year option extend year build m e maintenance
311281,311282,"Term Contract For Electrical Minor Works For Maintenance Of Lighting Systems In Vehicle Parks For A Period Of Three (3) Years With An Option To Extend For Two (2) Years Building, M&E Maintenance",term contract electrical minor work maintenance light system vehicle park period year option extend year build m e maintenance
311282,311283,Term Contract For Soil Investigation Works For A Period Of Two (2) Years With An Option To Extend For Another One (1) Year Soil Investigation & Stabilization,term contract soil investigation work period year option extend year soil investigation stabilization
311283,311284,Renewal Of Backup Software Licences Maintenance Softwares & Licences,renewal backup software licence maintenance software licence


https://www.quantstart.com/articles/Parallelising-Python-with-Threading-and-Multiprocessing

In [143]:
#!/usr/bin/env python3
# import itertools
# import multiprocessing
t=time()
#Generate values for each parameter
a = range(10)
b = range(10)
c = range(10)
d = range(10)

#Generate a list of tuples where each tuple is a combination of parameters.
#The list will contain all possible combinations of parameters.
paramlist = list(itertools.product(a,b,c,d))

#A function which will process a tuple of parameters
def func(params):
  a = params[0]
  b = params[1]
  c = params[2]
  d = params[3]
  return a*b*c*d

#Generate processes equal to the number of cores
pool = multiprocessing.Pool()

#Distribute the parameter sets evenly across the cores
res  = pool.map(func,paramlist)

print('Time to do mapping: {} mins'.format(round((time() - t) / 60, 2)))
type(res)

Time to do mapping: 0.0 mins


list

In [294]:
df_test = df2[:500]
df_test.shape

(500, 3)

In [77]:
df2.shape[0]/df_test.shape[0]*0.32/60

3.1062186666666673

In [169]:
multiprocessing.cpu_count()

4

In [174]:
#!/usr/bin/env python3
t = time()
# clusters_in_text = []
a = range(len(df_test))
b = range(len(df_word3))
paramlist = list(itertools.product(a,b))

# for j in a:
#     clusters = []
#     text = df_test.iloc[j,2]
#     if any(word in text.split() for word in df_word3.word_2):
# #         print(text)
#         for i in b:
#             word = df_word3.iloc[i,0]

#             if len(word.split())>1:
#                 if (word in text) & (df_word3.iloc[i,2]!='To Remove'):
#                     print(word, '|',text,'|', df_word3.iloc[i,2])
#                     clusters.append(df_word3.iloc[i,2])
#             else:   
#                 if (word in text.split()) & (df_word3.iloc[i,2]!='To Remove'):
#                     clusters.append(df_word3.iloc[i,2])

#     clusters = sorted(list(set(clusters)))
#     clusters_2 = ", ".join(str(v) for v in clusters)
#     clusters_in_text.append(clusters_2)

def func(params):
    j = params[0]
    text = df_test.iloc[j,2]
    
    i = params[1]
    word = df_word3.iloc[i,0]
    cluster = df_word3.iloc[i,2]
     
    if len(word.split())>1:
        if (word in text) & (cluster!='To Remove'):
#             print(word, '|',text,'|', cluster)
            return j, cluster
#             clusters.append(cluster)
    else:   
        if (word in text.split()) & (cluster!='To Remove'):
            return j, cluster
#             clusters.append(cluster)    
   
#Generate processes equal to the number of cores
pool = multiprocessing.Pool(multiprocessing.cpu_count())

#Distribute the parameter sets evenly across the cores
res  = pool.map(func,paramlist)

pool.close()
pool.join()


# df_test['cluster_set'] = clusters_in_text
print('Time to do mapping: {} mins'.format(round((time() - t) / 60, 2)))
# df_test
# set(res)

Time to do mapping: 0.24 mins


In [279]:
df_word3[df_word3.cluster_meaning=='To Remove'].head()

,word_2,cluster_3,cluster_meaning
4,jan,9999,To Remove
10,file,9999,To Remove
15,malan road,9999,To Remove
16,singapore,9999,To Remove
37,science,9999,To Remove


need for phrase senstive cluster detection is not strong since most phrases tend to be in same in the cluster as their root words

In [118]:
cluster_list = [2, 4, 5, 6, 9, 105, 109, 113]
text = 'provision design production delivery food nutrition teaching learning guide printing service'

for i in range(len(df_word2)):
    if (df_word2.iloc[i,0] in text.split()) & (df_word2.iloc[i,1] in cluster_list):
        print(df_word2.iloc[i,0], df_word2.iloc[i,1])

service 5
printing 105
delivery 4
production 9
provision 2
design 105
learning 109
teaching 109
food 113
guide 109
nutrition 6


In [74]:
test1=['program','training']
test1 = 'program training'.split()
test2=['prog','rain']

if any(word in test1 for word in test2):
    print('uff')

In [75]:
test1

['program', 'training']

In [ ]:
df_test